# Maps

You have been shown many ways to show maps in python.

In the first task where you have to show the location of the top 5 facilities 
you can use mplleaflet (as was done in the geospatial lecture lab), pylab baseline. You can also use a shapefile to show the zip contours, though the result will be less exciting.
In the following portions of the assignment, where you have to show the value of a variable by zipcode (cloropleth), you should use a shapefile and a color map corresponding to the values of the variable, as in HW 10.

## __Murder and Green space access metrics__: 

our recommandation is: 


find the N nearest locations (N nearest parks, N nearest precints, and N needs not be the same for parks and precints, use your judgment) to the center of each ZIP code; for each park divide the size of the park by the distance to the park, and sum over the parks selected. Similarly, for the murder rate: we found the N nearest precints to the center of the ZIP code, and summed the number of murders for each precint devided by the distance between the precint and the ZIP code center. Further, you may want to try the logarithm of the metrics described above, if you find that they vary too dramatically, to suppress their power. Other solutions can be found. Do whatever you think is most suitable, but describe and motivate what you decided to do, and what your metric of "access to green spaces" and "proximity to violent crimes" are.
   

## __311 calls__: 

you can aggregate the 311 calls at the zip code level directly (the zip code is available in the 311 calls data for every call).

some of the 311 calls may be directly related to asthma cases: complaints about construction, unsanitary conditions, mold, while others may not (illegally parked cars for example). This is a list of call descriptions which we considered relevant. You can modify this list as you see fit. But you must justify your choice of relevant complaints.

In [2]:
validcomplaints = ['UNSANITARY CONDITION','Dirty Conditions', 'Sewer',
                   'Hazardous Material','Construction',
                   'Indoor Air Quality','Indoor Sewage','Air Quality',
                   'HEATING','Industrial Waste','Mold','Asbestos',
                   'Sweeping/Missed-Inadequate','Smoking',
                   'Non-Residential Heat','Lead','Standing Water',
                    'Water Quality','Unsanitary Animal Facility',
                   'PAINT - PLASTER','GENERAL CONSTRUCTION']

Because complaints to 311 are issued at a different rate in different city areas (due to various reasons including a strong social bias by which wealthier people are generally more engaged and more likely to call) you must normalize the number of relevant calls (calls for one of the reasons in your list) by the total number of calls for that zip code (divide the number of relevant calls by the total number of calls) (normalize = divide)

## Further suggestions

### __Getting coordinates__:

shapefiles read in with geopanda objects as GeoDataFrame contain the attribute geometry:

gdf.geometry

that contains the attribute centroid

gdf.geometry.centroid

which (generally) contains an object of type POINT which is a set of 2 numbers: the longitude and latitude in our case. To extract longitude and latitude you can pass the gdf.geometry.centroid object and pass it to a function such as the one below

In [3]:
def getXY(pt):
    return pt.x, pt.y

### __ZIPCODE 83__: 
This may or may not be in your data, as it always happens with NYC data. It may mess things up. http://walk.allcitynewyork.com/2011/07/mysterious-case-of-ghost-zip-code.html
        you probably want to remove it (for example to constraint your nyczip files to zips larger than 10000)
        

### __ROCKAWAY 311 calls__ : 

we found a very large number of relevant calls after normalization in one of the Rockaway ZIP codes. While this may be due to a genuinely high rate of calls about the issues we identified as relevant (water related calls for example) the large number of calls in this ZIP code was skeweing our metric and we artificially suppressed it by replacing it with the mean number of calls. 

### __using the result of the regression as a metric__

Once you have the result of your regression you can use it as a metric for clustering. If you had a linear model for example, then you have something like: 

$$Asthma\_Rate = a*(murder\_rate)+b*(park\_access)+c*(highway\_length)+ ...$$

Then the coefficients for the $v$ variables: $coef(v)$ ($coef(murder\_rate) = a$, $coef(park\_access) = b$, $coef(highway\_length) = c$) can be used as weights in your distance metric. If you were using a Eucledian distance then the weighted distance $D_{12}$ between ZIP codes $Z_1$ and $Z_2$ would be

$$D_{12} = \sqrt{\sum_\mathrm{v\_in\_variables} (coef(v)^2*(Z_1(v) - Z_2(v))^2)}$$

summed over each $v$ in your model variables (in the case above murder rate, park access, highway length). If your model is more complex (higher degree linear, or non linear) then the distance D would use the variables as they enter in the model (eg the log of the distance if any was included in the regression through its log, a power exponent if you included any of them to any power, maybe a polynomial model for the variable etc etc).

Finally notice that since, variable by variable, the corresponding coefficient multiplies each datapoint, in the example above you would not have to define a separate affinity function (like we did in class talking about clustering): you can simply multiply each  data vector $Z(v)$ by the coefficient $coef(v)$ before you feed your data to the clustering algorithm that uses a euclidian distance.

## converting between coordinate systems
some of the files may be in different coordinate systems. 
to see the coordinate systems of a file you can call the .crs attribute:

In [ ]:
nyczips.crs

in case of naive geometry you may find this useful https://github.com/geopandas/geopandas/issues/245

to set a GoeDataFrame into the coordinate system of another GeoDataFrame you can do something like:

In [ ]:
pkgeo = parksdf.to_crs(nyczips.crs).geometry

and if you wish you can overwrite the geometry of the original file. This was shown to you in the geospatial lab.

## plotting

to plot the contours of a shape file (the zip boundaries) but not the cells in different colors use alpha = 0 as argument. 
to overplot two GeoDataFrames or a GeoDataFrame and a scatter plot pass an axes argument to the GeoDataFrame plot function

## preprocessing
remember to put your variables in the same order of magnitude before you run regression models, and to whiten the data before clustering

# clustering 
Use a "reasonable" number of clusters. You do not have to be too strict about evaluating the best number, but if you use K-means test for stability, and if you use agglomerative clustering check your dendrogram.